In [89]:
import numpy as np
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from mlxtend.plotting import plot_confusion_matrix
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import label_ranking_average_precision_score, label_ranking_loss
from sklearn.multioutput import MultiOutputClassifier



In [32]:
#read data
hr=pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
hr.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [33]:
hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [34]:
hr.isnull().sum().sort_values(ascending = False)

Age                         0
StandardHours               0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StockOptionLevel            0
MonthlyIncome               0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInCurrentRole          0
YearsSinceLastPromotion     0
MonthlyRate                 0
MaritalStatus               0
Attrition                   0
EmployeeCount               0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeNumber              0
JobSatisfaction             0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole   

In [35]:
hr['Attrition'].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [36]:
hr['PerformanceRating'].value_counts()

3    1244
4     226
Name: PerformanceRating, dtype: int64

In [37]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
newhr=hr.apply(LabelEncoder().fit_transform)


In [38]:
newhr['Attrition'].value_counts()

0    1233
1     237
Name: Attrition, dtype: int64

In [39]:
newhr.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,23,1,2,624,2,0,1,1,0,0,...,0,0,0,8,0,0,6,4,0,5
1,31,0,1,113,1,7,0,1,0,1,...,3,0,1,10,3,2,10,7,1,7
2,19,1,2,805,1,1,1,4,0,2,...,1,0,0,7,3,2,0,0,0,0
3,15,0,1,820,1,2,3,1,0,3,...,2,0,0,8,3,2,8,7,3,0
4,9,0,2,312,1,1,0,3,0,4,...,3,0,1,6,3,2,2,2,2,2


In [40]:
#splitting data to input and output
X=newhr.drop(['Attrition','PerformanceRating'],axis=1) #input
y = newhr[['Attrition','PerformanceRating']] #output(multi label)


In [41]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()  
X=scaler.fit_transform(X)    

In [42]:
from sklearn.model_selection import train_test_split    
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42)  

In [43]:
print(' X_train.shape : ',X_train.shape)
print(' y_train.shape : ',y_train.shape)
print(' X_test.shape : ',X_test.shape)
print(' y_test.shape : ',y_test.shape)

 X_train.shape :  (1176, 33)
 y_train.shape :  (1176, 2)
 X_test.shape :  (294, 33)
 y_test.shape :  (294, 2)


In [44]:
#model1
#knn

In [78]:
knn=KNeighborsClassifier()
knn = MultiOutputClassifier(knn)
pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, pred)
print('accuracy',avg_precision)



accuracy 0.9642857142857143


In [ ]:
#model2
#decision tree

In [79]:
dtree = DecisionTreeClassifier(ccp_alpha=0.01, criterion='entropy', max_depth=7,
                       max_features='auto')
multi_target_classifier = MultiOutputClassifier(dtree, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)


('accuracy:', 0.9455782312925171)

In [ ]:
#model 3
# random forest

In [80]:
random_forest = RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                       n_estimators=30)
multi_target_classifier = MultiOutputClassifier(random_forest, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9472789115646258)

In [103]:
#model 4
#support vector machine

In [119]:
svm=SVC(C=10,gamma=0.01)
multi_target_classifier = MultiOutputClassifier(svm,n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9608843537414966)

In [101]:
#model 5
#adaboost

In [127]:
adaboost = AdaBoostClassifier(n_estimators=20)
multi_target_classifier = MultiOutputClassifier(adaboost, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9591836734693877)

In [67]:
#model 5
#logistic regression

In [128]:
logistic =LogisticRegression(solver='newton-cg')
multi_target_classifier = MultiOutputClassifier(logistic, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9642857142857143)

In [ ]:
#model 6
#naive bias

In [84]:
NB = GaussianNB()
multi_target_classifier = MultiOutputClassifier(NB, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9727891156462585)

In [ ]:
#model 7
#gradient Boost

In [136]:
gboost = GradientBoostingClassifier(n_estimators=100)
multi_target_classifier = MultiOutputClassifier(gboost, n_jobs=-1)
Y_pred = multi_target_classifier.fit(X_train, y_train).predict(X_test)
avg_precision = label_ranking_average_precision_score(y_test, Y_pred)
('accuracy:',avg_precision)

('accuracy:', 0.9625850340136054)

In [ ]:
#model 1 ==> KNN , accuracy=0.9642857142857143
#model2 ==> Decision Tree , accuracy=0.9455782312925171
#model 3 ==> Random Forest ,accuracy=0.9472789115646258
#model 4 ==> SVM , accuracy=0.9608843537414966
#model 5 ==> adaboost , accuracy=0.9591836734693877
#model 6 ==> Logistic regression , accuracy=0.9642857142857143
#model 7 ==>Naive Biase , accuracy=0.9727891156462585
#model 8 ==>Gradient boost, accuracy= 0.9625850340136054


#best model is  Naive Biase with accuracy=0.9727891156462585